In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import os
mnist = input_data.read_data_sets("G:/dldata/MNIST_data/",one_hot=True)

Extracting G:/dldata/MNIST_data/train-images-idx3-ubyte.gz
Extracting G:/dldata/MNIST_data/train-labels-idx1-ubyte.gz
Extracting G:/dldata/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting G:/dldata/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#创建保存模型文件的目录
ckpt_dir = "G:/dldata/ckpt_dir/"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [3]:
#定义全连接层函数
def fcn_layer(inputs,             #输入数据
              input_dim,          #输入神经元数量
              output_dim,         #输出神经元数量
              activation=None):   #激活函数
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1)) #以截断正态分布的随机数初始化W
    b = tf.Variable(tf.zeros(output_dim))  #以0初始化b
    
    XWb = tf.matmul(inputs, W) + b  #j建立表达式：inputs * W + b
    
    if activation is None:  #默认不使用激活函数
        outputs = XWb    #若传入激活函数，则对输出结果进行变换
    else:
        outputs = activation(XWb)
    return outputs

In [4]:
#构建输入层
x = tf.placeholder(tf.float32, [None, 784], name="X")

In [5]:
###构建1隐层模型###

H1_NN = 256

#构建隐藏层1
h1 = fcn_layer(inputs = x,
               input_dim = 784,
               output_dim = H1_NN,
               activation = tf.nn.relu)
#构建输出层
forward = fcn_layer(inputs=h1,
                    input_dim=H1_NN,
                    output_dim=10,
                    activation=None)
pred = tf.nn.softmax(forward)

In [6]:
#定义标签数据占位符
y = tf.placeholder(tf.float32, [None, 10], name="Y")

In [7]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward, labels=y))

In [8]:
train_epochs = 40
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)
display_step = 1
learning_rate = 0.01

#存储模型的粒度
save_step = 5

In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

In [10]:
correct_prediction = tf.equal(tf.arg_max(pred, 1), tf.arg_max(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [11]:
#声明完所有变量后，通过saver存储模型
saver = tf.train.Saver()

from time import time
startTime = time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs,ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x:xs, y:ys})
        
    loss,acc = sess.run([loss_function,accuracy],
                        feed_dict={x:mnist.validation.images,
                                   y:mnist.validation.labels})
    if (epoch+1) % display_step == 0:
        print("Train Epoch:", '%02d'% (epoch+1), "loss=", "{:.9f}".format(loss),"Accuracy=","{:.4}".format(acc))
    
    if(epoch+1) % save_step == 0:
        saver.save(sess,os.path.join(ckpt_dir,'mnist_h256_model_{:06d}.ckpt'.format(epoch+1)))
        print('mnist_h256_model_{:06d}.ckpt saved'.format(epoch+1))

saver.save(sess, os.path.join(ckpt_dir,'mnist_h256_model.ckpt'))       
print("Model saved!")

duration = time() - startTime
print("Train Finished takes:","{:.2f}".format(duration))

Train Epoch: 01 loss= 0.154859662 Accuracy= 0.9552
Train Epoch: 02 loss= 0.183831990 Accuracy= 0.9512
Train Epoch: 03 loss= 0.139997780 Accuracy= 0.9624
Train Epoch: 04 loss= 0.182067156 Accuracy= 0.9578
Train Epoch: 05 loss= 0.159927368 Accuracy= 0.9666
mnist_h256_model_000005.ckpt saved
Train Epoch: 06 loss= 0.170264184 Accuracy= 0.9628
Train Epoch: 07 loss= 0.223159358 Accuracy= 0.963
Train Epoch: 08 loss= 0.230628178 Accuracy= 0.965
Train Epoch: 09 loss= 0.204531595 Accuracy= 0.9674
Train Epoch: 10 loss= 0.218626246 Accuracy= 0.9684
mnist_h256_model_000010.ckpt saved
Train Epoch: 11 loss= 0.201592609 Accuracy= 0.9716
Train Epoch: 12 loss= 0.232062131 Accuracy= 0.9724
Train Epoch: 13 loss= 0.248739630 Accuracy= 0.9698
Train Epoch: 14 loss= 0.265724599 Accuracy= 0.9726
Train Epoch: 15 loss= 0.206563190 Accuracy= 0.973
mnist_h256_model_000015.ckpt saved
Train Epoch: 16 loss= 0.294457555 Accuracy= 0.9702
Train Epoch: 17 loss= 0.296203017 Accuracy= 0.968
Train Epoch: 18 loss= 0.32333776